In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

import pandas as pd

In [2]:
  query = f"""select "VALUE1",  "PUBLICATION_DATE", "PERIOD_TO"
  from "RISKACCESS"."XXMR_MADAB_CONTENT"
    where "RISKACCESS"."XXMR_MADAB_CONTENT"."COMMODITY_ID" = 2332
    and "PUBLICATION_DATE" >= TO_DATE('2022-01-01', 'YYYY-MM-DD')
"""
data= export_from_RISKCUSTOM(query)
# data.to_excel('USD-BRL_2022-2024.xlsx')

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [23]:
"""with params as (
  select date '2023-12-31' ""onDate"" from dual
),
base as (
  select
	    ""onDate"",
	    i.""counterpartyId"",
	    ""counterpartyCountryName"",
	    ""buCode"",
	    ""fiDocumentNumber"",
	    ""documentDate"",
	    ""dueDate"",
	    ""reportDate"",
	    ""documentCurrency"",
	    ""creditId"",
	    ""baseCreditId"",
	    ""inn"",
	    max(i.""counterpartyName"")   ""counterpartyName"",
	    max(""buName"")             ""buName"",
	    sum(""debtInDocCurrency"")  ""debtInDocCurrency""
    from ""rmOverdueInvoicesOgs"" i
    left join ""mdgCounterpartyLimitCategory"" mclc 
		on mclc.""counterpartyId"" = i.""counterpartyId""
	left join ""mdgOverdueSettings"" mos 
		on mos.""charValue2""  = i.""counterpartyId"" 
	   and mos.""charValue1"" = i.""buCode""
	   and mos.""code"" = 'BI_INTRGR_EXCLUDE'
	left join xxmdg.""salesUnits""      su1 on (su1.""id"" = i.""buCode"")
	left join xxmdg.""salesUnits""      su2 on (su2.""oebs12ShortCode"" = i.""buCode"")
	left join xxmdg.""salesUnits""      su3 on (su3.""sapId"" = i.""buCode"")
    cross join params p
  where 1=1
  and coalesce(su1.""businessSegment"", su2.""businessSegment"", su3.""businessSegment"") = 'EUROCHEM'
  and ""rowType"" = 'D'
  --and coalesce(ISSUE_TYPE, 'C') not in ('C')
  and ""documentDate"" >= date '2023-01-01'
  and ""documentDate"" < date '2024-01-01'
  and ""accountTypeName"" = 'Trade receivables'
  and ""unpaidAmount"" >= 0
  and ""debtInDocCurrency"" >= 0
  and ""reportDate"" <= ""onDate""
  and mclc.""##batch_id"" is null
  and mos.""##batch_id"" is null
  --and ""buCode"" = 'BRHRG'
--  and ""buCode"" = '6700' and ""counterpartyId"" = '1000004243' -- test strange pd
  group by
    ""onDate"",
    i.""counterpartyId"",
    ""counterpartyCountryName"",
    ""buCode"",
    ""fiDocumentNumber"",
    ""documentDate"",
    ""dueDate"",
    ""reportDate"",
    ""documentCurrency"",
    ""creditId"",
    ""baseCreditId"",
    ""inn""
),
c1 as (
  select
    i.*,
    ""debtInDocCurrency"" - lag(""debtInDocCurrency"", 1, 0)
      over(partition by ""buCode"", ""counterpartyId"", ""fiDocumentNumber"", ""documentDate"", ""documentCurrency"", ""dueDate"" order by ""reportDate"") ""debtInDocCurrencyDiff"",
    r.""rate""                       ""fxRateUsd"",
    r.""rate"" * ""debtInDocCurrency"" ""debtInUsd""
    from base i
    left join ""currencyRates"" r  on (r.""type"" = 'Actual' and r.""to"" = 'USD' and r.""from"" = i.""documentCurrency"" and r.""date"" = ""onDate"")
),
c2 as (
  select
    c.*,
    case when row_number()
      over(partition by ""buCode"", ""counterpartyId"", ""fiDocumentNumber"", ""documentDate"", ""documentCurrency"", ""dueDate"" order by ""reportDate"") = 1 then 'Y' end ""firstRowFlag"",
    case when row_number()
      over(partition by ""buCode"", ""counterpartyId"", ""fiDocumentNumber"", ""documentDate"", ""documentCurrency"", ""dueDate"" order by ""reportDate"" desc) = 1 then 'Y' end ""lastRowFlag"",
    last_value(coalesce(""baseCreditId"", ""creditId"") ignore nulls)
      over(partition by ""buCode"", ""counterpartyId"" order by ""reportDate"" desc) ""creditIdUpd""
  from c1 c
  where ""debtInDocCurrencyDiff"" != 0 or ""reportDate"" = ""onDate""
),
c3 as (
  select
    ""onDate"",
    ""counterpartyName"",
    ""counterpartyId"",
    ""counterpartyCountryName"",
    ""buCode"",
    ""buName"",
    ""fiDocumentNumber"",
    ""documentDate"",
    ""dueDate"",
    case ""firstRowFlag"" when 'Y' then ""documentDate"" else ""reportDate"" end ""changeDate"",
    ""documentCurrency"",
    ""debtInDocCurrency"",
    ""debtInDocCurrencyDiff"",
    ""fxRateUsd"",
    ""debtInUsd"",
    ""debtInDocCurrencyDiff"" * ""fxRateUsd"" ""debtInUsdDiff"",
    ""creditIdUpd"",
    ""inn""
  from c2 c
  union all
  select
    ""onDate"",
    ""counterpartyName"",
    ""counterpartyId"",
    ""counterpartyCountryName"",
    ""buCode"",
    ""buName"",
    ""fiDocumentNumber"",
    ""documentDate"",
    ""dueDate"",
    ""reportDate"" + 1 ""changeDate"",
    ""documentCurrency"",
    0 ""debtInDocCurrency"",
    -""debtInDocCurrency"" ""debtInDocCurrencyDiff"",
    ""fxRateUsd"",
    0 ""debtInUsd"",
    -""debtInDocCurrencyDiff"" * ""fxRateUsd"" ""debtInUsdDiff"",
    ""creditIdUpd"",
    ""inn""
  from c2 c
  where c.""lastRowFlag"" = 'Y'
  and ""reportDate"" < ""onDate""
)
--select count(*), count(""pd""), count(""pdToday"") from (select ""counterpartyId"", max(""pd"") ""pd"", max(""pdToday"") ""pdToday"" from c3 group by ""counterpartyId"");
--/
select
  c3.""counterpartyName"",
  ""counterpartyId"",
  ""counterpartyCountryName"",
  ""buCode"",
  ""buName"",
  ""fiDocumentNumber"",
  ""documentDate"",
  ""dueDate"",
  ""changeDate"",
  ""dueDate"" - ""changeDate"" ""daysOverdue"",
  ""documentCurrency"",
  ""debtInDocCurrency"",
  ""debtInDocCurrencyDiff"",
  ""fxRateUsd"",
  ""debtInUsd"",
  ""debtInUsdDiff"",
  coalesce(pt.""rating"", rm_overdue_invoices_ogs_pkg.camRating(""changeDate"", substr(""creditIdUpd"", 6))) as ""ratingChangeDate"",
  rm_overdue_invoices_ogs_pkg.camPd(""changeDate"", substr(""creditIdUpd"", 6)) as ""pdChangeDate""
--  getPd(""changeDate"", substr(""creditIdUpd"", 6)) ""pdOnChangeDate"",
--  getPd(trunc(sysdate), substr(""creditIdUpd"", 6)) ""pdToday""
--  ,""creditIdUpd""
from c3
  left join ""agrometrikaProposalTrackingCounterparty"" pt on (
    ""buCode"" in ('BRHRG', 'BRFTO', 'BRSFT')
    and pt.""holding"" = 'EUROCHEM'
    and pt.""cpfCnpj"" = c3.""inn""
    and c3.""changeDate"" between pt.""limit_valid_from"" and pt.""limit_valid_until""
)
order by ""buCode"", ""counterpartyId"", ""fiDocumentNumber"", ""documentDate"", ""documentCurrency"", ""changeDate""
"""


,1
##batch_id,2
##inserted,2
rowType,2
amount,2
docAmount,2
accountedAmount,2
accountedDocAmount,2
documentDate,21
dueDate,14
invoiceNum,21


In [39]:
data_export['Index_copy'] = data_export.index
group_data = data_export.pivot_table(index=['customerName', 'internalExternalEN', 'orgName', 'dueDate','presentAmountUSD', 'dealNumber', 'holding', 'currencyCode', 'sourceSystem', 'bscCode'], values='Index_copy', aggfunc={'Index_copy': list}).reset_index()
group_data 

,customerName,internalExternalEN,orgName,dueDate,presentAmountUSD,dealNumber,holding,currencyCode,sourceSystem,bscCode,Index_copy
0,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-04-15,141.00,4000184460,SUEK,RUB,SAP ERP,BSC135,"[12, 35]"
1,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-04-15,198.68,4000184460,SUEK,RUB,SAP ERP,BSC135,"[13, 15, 16, 17, 18, 19, 20, 33, 34, 36, 37, 3..."
2,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-05-15,198.68,4000184460,SUEK,RUB,SAP ERP,BSC135,"[8, 41]"
3,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-06-15,198.68,4000184460,SUEK,RUB,SAP ERP,BSC135,"[9, 26]"
4,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-07-15,198.68,4000184460,SUEK,RUB,SAP ERP,BSC135,"[11, 25]"
5,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-08-15,198.68,4000184460,SUEK,RUB,SAP ERP,BSC135,"[10, 32]"
6,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-09-15,198.68,4000184460,SUEK,RUB,SAP ERP,BSC135,"[0, 27]"
7,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-10-15,198.68,4000184460,SUEK,RUB,SAP ERP,BSC135,"[5, 24]"
8,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-11-15,198.68,4000184460,SUEK,RUB,SAP ERP,BSC135,"[4, 22]"
9,"ООО ""СУЭК""",Internal,"АО ""СУЭК""",2023-12-15,198.68,4000184460,SUEK,RUB,SAP ERP,BSC135,"[6, 21]"


In [54]:
data_same = data_export[data_export.dueDate == '2023-04-15']
 
# data_same = data_same[(data_same["sourceSystem"] == 'OEBS12_EU') | (data_same["sourceSystem"] == 'ETL document loader')]
data_same = data_same[data_same["rowType"] == 'M']
column_dict = {}
for column in data_same.columns.tolist():
    column_dict[column] = len(data_same[column].unique().tolist())
column_df = pd.DataFrame(column_dict, index=[1]).transpose()
column_df[column_df.iloc[:, 0] > 1]
# data_same.amount.unique()
# data_same.shape

,1
amount,2
docAmount,2
accountedAmount,2
accountedDocAmount,2
documentDate,8
invoiceNum,8
transactionNumber,8
glDate,2
amountFunctional,2
amountDocFunctional,2
